In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import csbgnpy.pd.io.sbgnml
import pypint

This notebook has been executed using the docker image `pauleve/clockcycle:v0`

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [3]:
merge_file = "generated/MERGE.sbgnml"
merge = csbgnpy.pd.io.sbgnml.read(merge_file)

## Phases and markers

From [2 - Markers](2%20-%20Markers.ipynb)

In [4]:
%store -r cycle_markers
%store -r cycle_phases

To ids:

In [5]:
markers = {phase: [merge.get_entity(e, by_string=True).id for e in ms]
               for phase, ms in cycle_markers.items()}

## Initial state

From [3 - Initial state](3%20-%20Initial%20state.ipynb)

In [6]:
%store -r ini

To ids:

In [7]:
ini = [e.id for e in ini]

## Dynamical model

In [8]:
an = pypint.load(merge_file, initial_state=ini)

## A. Reach each phase

In [9]:
for phase in markers:
    res = []
    for marker in markers[phase]:
        res.append(an.reachability((marker, 1), fallback="mole"))
    print(phase, "{}/{}".format(sum(res), len(res)))

G0 2/2
earlyG1 5/5
lateG1 2/2
earlyS 1/1
lateS 1/1
G2 4/4
M 4/4


## B. Reach each phase with other phases to zero

In [10]:
for phase in markers:
    res = []
    for marker in markers[phase]:
        l = [(marker, 1)]
        for other_phase in markers:
            if other_phase != phase:
                l += [(marker2, 0) for marker2 in markers[other_phase]]
        res.append(an.reachability((marker, 1), fallback="mole"))
    print(phase, "{}/{}".format(sum(res), len(res)))        

G0 2/2
earlyG1 5/5
lateG1 2/2
earlyS 1/1
lateS 1/1
G2 4/4
M 4/4


## C. Reach each phase with previous phase(s) disabled

In [11]:
couples = [(["earlyG1"], "lateG1"), (["lateG1"], "earlyS"), (["earlyS"], "lateS"), (["lateS"], "G2"), (["G2"], "M"), (["M"], "earlyG1")]
for phases_disabled, phase in couples:
    res = []
    an_disabled = an.disable([(marker, 1) for phase_disabled in phases_disabled for marker in markers[phase_disabled]])
    for marker in markers[phase]:
        res.append(an_disabled.reachability((marker, 1), fallback="mole"))
    print(", ".join(phases_disabled), "->", phase, "{}/{}".format(sum(res), len(res)))    

earlyG1 -> lateG1 0/2
lateG1 -> earlyS 0/1
earlyS -> lateS 0/1
lateS -> G2 4/4
G2 -> M 0/4
M -> earlyG1 5/5
